In [1]:
import requests
import warnings
import os

import pandas as pd

from pymisp import PyMISP, MISPObject, MISPAttribute
from bs4 import BeautifulSoup
from dotenv import load_dotenv

In [2]:
load_dotenv()

misp_api_key = os.getenv("MISP_API")

In [3]:
securityBulletin = pd.DataFrame(columns=('RefernceN', 'Title', 'ArticleLink'))

# The first 4 pages
for pageN in range(0, 4):
    URL = f"https://www.dgssi.gov.ma/fr/macert/bulletins-de-securite.html/page/{pageN}"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.findAll('a', class_="title_event")

    resultsDF = pd.DataFrame(
        ([title.text.strip().split('-')[0], "".join(title.text.strip().split('-')[1:]), "https://www.dgssi.gov.ma" + title.find('a')['href']] for title in results),
        columns=('RefernceN', 'Title', 'ArticleLink')
    )

    securityBulletin = pd.concat([securityBulletin, resultsDF], ignore_index=True)

securityBulletin

,RefernceN,Title,ArticleLink
0,40770903/23,Exploitation d’une faille affectant « Zoho Ma...,https://www.dgssi.gov.ma/fr/content/4077090323...
1,40760903/23,Vulnérabilités dans Apache Tomcat,https://www.dgssi.gov.ma/fr/content/4076090323...
2,40750903/23,Vulnérabilités affectant le navigateur Googl...,https://www.dgssi.gov.ma/fr/content/4075090323...
3,40740903/23,Vulnérabilités affectant Cisco IOS XR Software,https://www.dgssi.gov.ma/fr/content/4074090323...
4,40720903/23,Vulnérabilité dans Veeam Backup,https://www.dgssi.gov.ma/fr/content/4072090323...
5,40730903/23,Vulnérabilités dans les produits Fortinet,https://www.dgssi.gov.ma/fr/content/4073090323...
6,40700803/23,Vulnérabilités affectant le système d’exploit...,https://www.dgssi.gov.ma/fr/content/4070080323...
7,40710803/23,Exploitation d’une faille critique dans Micro...,https://www.dgssi.gov.ma/fr/content/4071080323...
8,40680703/23,Vulnérabilités critiques affectant le thème «...,https://www.dgssi.gov.ma/fr/content/4068070323...
9,40690703/23,Vulnérabilités dans les produits IBM,https://www.dgssi.gov.ma/fr/content/4069070323...


In [6]:
def add_group_fromDF(MISPInstance, eventID, df):
    for i, row in df.iterrows():
        name = row["RefernceN"] + " " + row["Title"]
        object = MISPObject("blog", name=name, standalone=False)

        object.add_attribute("title", value=row["Title"], type="text")
        object.add_attribute("link", value=row["ArticleLink"], type="link")
        
        MISPInstance.add_object(eventID, object)

In [7]:
warnings.filterwarnings("ignore")

## Ignoring the certificate verification

misp_url = "https://localhost/"
misp_key = misp_api_key
misp_verifycert = False

myMISPInstance = PyMISP( misp_url, misp_key, misp_verifycert, "json" )

## Example
add_group_fromDF(myMISPInstance, "6", securityBulletin)